In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
gdp_data = pd.read_csv('/kaggle/gdp_csv.csv')
gdp_data

,Country Name,Country Code,Year,Value
0,Arab World,ARB,1968,2.576068e+10
1,Arab World,ARB,1969,2.843420e+10
2,Arab World,ARB,1970,3.138550e+10
3,Arab World,ARB,1971,3.642691e+10
4,Arab World,ARB,1972,4.331606e+10
...,...,...,...,...
11502,Zimbabwe,ZWE,2012,1.424249e+10
11503,Zimbabwe,ZWE,2013,1.545177e+10
11504,Zimbabwe,ZWE,2014,1.589105e+10
11505,Zimbabwe,ZWE,2015,1.630467e+10


In [ ]:
#loading 2020 data
athletes_2020 = pd.read_excel('/kaggle/Athletes_2020.xlsx')
# Coaches_2020 = pd.read_excel('/kaggle/Coaches_2020.xlsx')
Medals_2020 = pd.read_excel('/kaggle/Medals_2020.xlsx')
Teams_2020 = pd.read_excel('/kaggle/Teams_2020.xlsx')
# athletes_2020.head()
# name, discipline, NOC, team

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
#loading 2024 data
# events_2024 = pd.read_csv('/kaggle/events_2024.csv')
# schedules_2024 = pd.read_csv('/kaggle/schedules_2024.csv')
# venues_2024 = pd.read_csv('/kaggle/venues_2024.csv')
# torch_route_2024 = pd.read_csv('/kaggle/torch_route_2024.csv')
# schedules_2024.head()


In [ ]:
#loading the historic data
historic_atheletes = pd.read_csv('/kaggle/athlete_events_historic.csv')
#historic_noc_regions = pd.read_csv('/kaggle/noc_regions_historic.csv')
# historic_atheletes.head()

In [ ]:
data = historic_atheletes[historic_atheletes['Season'] == 'Summer']

data = data.drop(columns=['Season', 'Event', 'Games'])
data['Medal'] = data['Medal'].fillna('No Medals')
data.head()
# data[(data['Team'] == 'United States') & (data['Medal'].isin(['Gold', 'Silver']))]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Year,City,Sport,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992,Barcelona,Basketball,No Medals
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012,London,Judo,No Medals
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920,Antwerpen,Football,No Medals
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900,Paris,Tug-Of-War,Gold
26,8,"Cornelia ""Cor"" Aalten (-Strannood)",F,18.0,168.0,NaN,Netherlands,NED,1932,Los Angeles,Athletics,No Medals


In [ ]:
# create host country column

host_countries = {
    1896: 'Greece',
    1900: 'France',
    1904: 'USA',
    1906: 'Greece',
    1908: 'UK',
    1912: 'Sweden',
    1920: 'Belgium',
    1924: 'France',
    1928: 'Netherlands',
    1932: 'USA',
    1936: 'Germany',
    1940: 'Japan/Finland',  # Note: Combined host cities
    1944: 'UK',             # Not held
    1948: 'UK',
    1952: 'Finland',
    1956: 'Australia',      # Note: Also held in Sweden
    1960: 'Italy',
    1964: 'Japan',
    1968: 'Mexico',
    1972: 'Germany',
    1976: 'Canada',
    1980: 'USSR',
    1984: 'USA',
    1988: 'South Korea',
    1992: 'Spain',
    1996: 'USA',
    2000: 'Australia',
    2004: 'Greece',
    2008: 'China',
    2012: 'UK',
    2016: 'Brazil',
    2021: 'Japan'
}


years = list(range(1896, 2022))

data = data.drop(columns=['City', 'ID', 'Name', 'Sex', 'Height', 'Weight'])
# Map the host countries based on the 'Year' column
data['Host_Country'] = data['Year'].map(host_countries)
data

,Age,Team,NOC,Year,Sport,Medal,Host_Country
0,24.0,China,CHN,1992,Basketball,No Medals,Spain
1,23.0,China,CHN,2012,Judo,No Medals,UK
2,24.0,Denmark,DEN,1920,Football,No Medals,Belgium
3,34.0,Denmark/Sweden,DEN,1900,Tug-Of-War,Gold,France
26,18.0,Netherlands,NED,1932,Athletics,No Medals,USA
...,...,...,...,...,...,...,...
271106,27.0,Argentina,ARG,2004,Hockey,No Medals,Greece
271107,21.0,United States,USA,1972,Football,No Medals,Germany
271108,24.0,Russia,RUS,2000,Rowing,No Medals,Australia
271109,28.0,Russia,RUS,2004,Rowing,No Medals,Greece


In [ ]:
# get rid of age column and use average age by country and year
data['Average_Age'] = data.groupby(['Year', 'Team'])['Age'].transform('mean')

# Drop the original Age column
data.drop(columns=['Age'], inplace=True)
data['Average_Age'] = data['Average_Age'].round(2)
data['Year'].min()

1896

In [ ]:
country_population = pd.read_excel('/kaggle/country_population_data.xlsx')


In [ ]:
population_data = country_population[(country_population['Year'] > 1900) & (country_population['Year'] < 2024)]
population_data

,Area,Year,Population,Population with interpolations,Source,source - details (col 1),source details (col 2),Available on-line,Type of primary data,Data footnote,...,Gapminder modifications - explanations col. 2,Other notes by gapminder (col 1).,Other notes by gapminder (col 2).,Geographical coverage (col 1),Geographical coverage (col 2),Secondary layer of sources (col 1),Secondary layer of sources (col 2),Tertiary layer of sources (col 1),Tertiary layer of sources (col 2),Tertiary layer of sources (col 3)
304,Abkhazia,1901,NaN,NaN,No data,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,Abkhazia,1902,NaN,NaN,No data,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,Abkhazia,1903,NaN,NaN,No data,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,Abkhazia,1904,NaN,NaN,No data,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,Abkhazia,1905,NaN,NaN,No data,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131547,Åland,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131548,Åland,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131549,Åland,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131550,Åland,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
population_data = country_population[(country_population['Year'] > 1890) & (country_population['Year'] < 2024)][['Year', 'Area', 'Population with interpolations']]

# Merge population_data with main_data based on Year and Country
population_data

# Display the merged dataset

,Year,Area,Population with interpolations
294,1891,Abkhazia,NaN
295,1892,Abkhazia,NaN
296,1893,Abkhazia,NaN
297,1894,Abkhazia,NaN
298,1895,Abkhazia,NaN
...,...,...,...
131547,2019,Åland,NaN
131548,2020,Åland,NaN
131549,2021,Åland,NaN
131550,2022,Åland,NaN


In [ ]:
population_data.rename(columns={'Area': 'Team'}, inplace=True)
population_data

,Year,Team,Population with interpolations
294,1891,Abkhazia,NaN
295,1892,Abkhazia,NaN
296,1893,Abkhazia,NaN
297,1894,Abkhazia,NaN
298,1895,Abkhazia,NaN
...,...,...,...
131547,2019,Åland,NaN
131548,2020,Åland,NaN
131549,2021,Åland,NaN
131550,2022,Åland,NaN


In [ ]:
merged_data = pd.merge(data, population_data, on=['Year', 'Team'], how='left')
merged_data.rename(columns={'Population with interpolations': 'Population'}, inplace=True)
merged_data

,Team,NOC,Year,Sport,Medal,Host_Country,Average_Age,Population
0,China,CHN,1992,Basketball,No Medals,Spain,21.96,1.175230e+09
1,China,CHN,2012,Judo,No Medals,UK,23.89,1.353601e+09
2,Denmark,DEN,1920,Football,No Medals,Belgium,28.83,3.242000e+06
3,Denmark/Sweden,DEN,1900,Tug-Of-War,Gold,France,30.83,NaN
4,Netherlands,NED,1932,Athletics,No Medals,USA,37.56,8.123000e+06
...,...,...,...,...,...,...,...,...
222547,Argentina,ARG,2004,Hockey,No Medals,Greece,27.01,3.834078e+07
222548,United States,USA,1972,Football,No Medals,Germany,23.62,2.132504e+08
222549,Russia,RUS,2000,Rowing,No Medals,Australia,25.23,1.467575e+08
222550,Russia,RUS,2004,Rowing,No Medals,Greece,26.05,1.443070e+08


In [ ]:
historic_noc = pd.read_csv('/kaggle/noc_regions_historic.csv')
data = pd.DataFrame(historic_noc)
data

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN
...,...,...,...
225,YEM,Yemen,NaN
226,YMD,Yemen,South Yemen
227,YUG,Serbia,Yugoslavia
228,ZAM,Zambia,NaN


In [ ]:
gdp_data.rename(columns={'Country Name': 'Team'}, inplace=True)
merged_data = pd.merge(merged_data, gdp_data, on=['Year', 'Team'], how='left')
merged_data.rename(columns={'Value': 'GDP'}, inplace=True)
merged_data = merged_data.drop(columns=['Country Code'])

In [ ]:
merged_data

,Team,NOC,Year,Sport,Medal,Host_Country,Average_Age,Population,GDP
0,China,CHN,1992,Basketball,No Medals,Spain,21.96,1.175230e+09,4.269157e+11
1,China,CHN,2012,Judo,No Medals,UK,23.89,1.353601e+09,8.560547e+12
2,Denmark,DEN,1920,Football,No Medals,Belgium,28.83,3.242000e+06,NaN
3,Denmark/Sweden,DEN,1900,Tug-Of-War,Gold,France,30.83,NaN,NaN
4,Netherlands,NED,1932,Athletics,No Medals,USA,37.56,8.123000e+06,NaN
...,...,...,...,...,...,...,...,...,...
222547,Argentina,ARG,2004,Hockey,No Medals,Greece,27.01,3.834078e+07,1.646579e+11
222548,United States,USA,1972,Football,No Medals,Germany,23.62,2.132504e+08,1.282449e+12
222549,Russia,RUS,2000,Rowing,No Medals,Australia,25.23,1.467575e+08,NaN
222550,Russia,RUS,2004,Rowing,No Medals,Greece,26.05,1.443070e+08,NaN


In [ ]:
unemployment = pd.read_excel('/kaggle/unemployment.xlsx')
data = pd.DataFrame(unemployment)
data.drop(columns=data.loc[:, 'Country Code':'1969'], inplace=True)
data

,Country Name,1970,1971,1972,1973,1974,1975,1976,1977,1978,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.915000,NaN,NaN,11.184000,NaN,NaN,11.710,5.580,NaN,NaN
3,Africa Western and Central,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.356261,NaN,NaN,6.991943,NaN,7.793947,NaN,NaN,3.289223,NaN
4,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.579000,NaN,NaN,NaN,NaN,16.497000,NaN,15.799,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.264000,32.840,27.491,30.344000,27.761,25.069000,25.459,20.392,12.093000,NaN
262,"Yemen, Rep.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.467000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,South Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.606000,22.869,24.023,23.992000,24.218,25.538000,24.339,28.770,28.838000,27.989
264,Zambia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,11.888,NaN,8.518000,5.031,5.539000,6.032,5.195,5.993000,NaN


In [ ]:
unemployment_data = data.melt(id_vars=['Country Name'], var_name='Year', value_name='Unemployment')
unemployment_data

,Country Name,Year,Unemployment
0,Aruba,1970,NaN
1,Africa Eastern and Southern,1970,NaN
2,Afghanistan,1970,NaN
3,Africa Western and Central,1970,NaN
4,Angola,1970,NaN
...,...,...,...
14359,Kosovo,2023,NaN
14360,"Yemen, Rep.",2023,NaN
14361,South Africa,2023,27.989
14362,Zambia,2023,NaN


In [ ]:
unemployment_data.rename(columns={'Country Name': 'Team'}, inplace=True)
merged_data['Year'] = merged_data['Year'].astype(int)
merged_data['Team'] = merged_data['Team'].astype(str)
unemployment_data['Year'] = unemployment_data['Year'].astype(int)
unemployment_data['Team'] = unemployment_data['Team'].astype(str)
merged_data = pd.merge(merged_data, unemployment_data, on=['Year', 'Team'], how='left')
merged_data = merged_data[(merged_data['Year'] > 1959)]


In [ ]:
merged_data.drop(columns=['Unemployment_y'], inplace=True)


In [ ]:
merged_data.rename(columns={'Unemployment_x': 'Unemployment'}, inplace=True)
merged_data

,Team,Year,Sport,Medal,Host_Country,Average_Age,Population,GDP,Unemployment
0,China,1992,Basketball,No Medals,Spain,21.96,1.175230e+09,4.269157e+11,2.300
1,China,2012,Judo,No Medals,UK,23.89,1.353601e+09,8.560547e+12,4.100
2,Finland,2000,Badminton,No Medals,Australia,26.22,5.173370e+06,1.255399e+11,11.135
3,Finland,1996,Sailing,No Medals,USA,26.03,5.125177e+06,1.320994e+11,15.572
4,Finland,2000,Sailing,No Medals,Australia,26.22,5.173370e+06,1.255399e+11,11.135
...,...,...,...,...,...,...,...,...,...
166262,Argentina,2004,Hockey,No Medals,Greece,27.01,3.834078e+07,1.646579e+11,13.522
166263,United States,1972,Football,No Medals,Germany,23.62,2.132504e+08,1.282449e+12,5.600
166264,Russia,2000,Rowing,No Medals,Australia,25.23,1.467575e+08,NaN,NaN
166265,Russia,2004,Rowing,No Medals,Greece,26.05,1.443070e+08,NaN,NaN


In [ ]:
merged_data.drop(columns=['NOC'], inplace=True)

KeyError: "['NOC'] not found in axis"

In [ ]:
total = merged_data.isna().any(axis=1).sum()
total

38466

In [ ]:
average_population = merged_data['Population'].mean()
average_GDP = merged_data['GDP'].mean()
average_Unemployment = merged_data['Unemployment'].mean()
average_Age = merged_data['Average_Age'].mean()
# Replace NaN values with the calculated average population
merged_data['Population'] = merged_data['Population'].fillna(average_population)
merged_data['GDP'] = merged_data['GDP'].fillna(average_GDP)
merged_data['Unemployment'] = merged_data['Unemployment'].fillna(average_Unemployment)
merged_data['Average_Age'] = merged_data['Average_Age'].fillna(average_Age)
merged_data

,Team,Year,Sport,Medal,Host_Country,Average_Age,Population,GDP,Unemployment
0,China,1992,Basketball,No Medals,Spain,21.96,1.175230e+09,4.269157e+11,2.300000
1,China,2012,Judo,No Medals,UK,23.89,1.353601e+09,8.560547e+12,4.100000
2,Finland,2000,Badminton,No Medals,Australia,26.22,5.173370e+06,1.255399e+11,11.135000
3,Finland,1996,Sailing,No Medals,USA,26.03,5.125177e+06,1.320994e+11,15.572000
4,Finland,2000,Sailing,No Medals,Australia,26.22,5.173370e+06,1.255399e+11,11.135000
...,...,...,...,...,...,...,...,...,...
166262,Argentina,2004,Hockey,No Medals,Greece,27.01,3.834078e+07,1.646579e+11,13.522000
166263,United States,1972,Football,No Medals,Germany,23.62,2.132504e+08,1.282449e+12,5.600000
166264,Russia,2000,Rowing,No Medals,Australia,25.23,1.467575e+08,1.171604e+12,7.699617
166265,Russia,2004,Rowing,No Medals,Greece,26.05,1.443070e+08,1.171604e+12,7.699617


In [ ]:
total = merged_data.isna().any(axis=1)
print(merged_data[total])

Empty DataFrame
Columns: [Team, Year, Sport, Medal, Host_Country, Average_Age, Population, GDP, Unemployment]
Index: []


In [ ]:
# One hot encoding for categorical variables
encoded_data = pd.get_dummies(merged_data, columns=['Host_Country', 'Team', 'Medal', 'Sport'])
encoded_data

,Year,Average_Age,Population,GDP,Unemployment,Host_Country_Australia,Host_Country_Brazil,Host_Country_Canada,Host_Country_China,Host_Country_Germany,...,Sport_Synchronized Swimming,Sport_Table Tennis,Sport_Taekwondo,Sport_Tennis,Sport_Trampolining,Sport_Triathlon,Sport_Volleyball,Sport_Water Polo,Sport_Weightlifting,Sport_Wrestling
0,1992,21.96,1.175230e+09,4.269157e+11,2.300000,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2012,23.89,1.353601e+09,8.560547e+12,4.100000,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2000,26.22,5.173370e+06,1.255399e+11,11.135000,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1996,26.03,5.125177e+06,1.320994e+11,15.572000,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2000,26.22,5.173370e+06,1.255399e+11,11.135000,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166262,2004,27.01,3.834078e+07,1.646579e+11,13.522000,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
166263,1972,23.62,2.132504e+08,1.282449e+12,5.600000,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
166264,2000,25.23,1.467575e+08,1.171604e+12,7.699617,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
166265,2004,26.05,1.443070e+08,1.171604e+12,7.699617,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
features = encoded_data.drop(columns=['Medal_No Medals', 'Medal_Gold', 'Medal_Silver', 'Medal_Bronze'])
target = encoded_data[['Medal_Gold', 'Medal_Silver', 'Medal_Bronze', 'Medal_No Medals']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
# DONT RUN THIS

model = RandomForestRegressor(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

In [ ]:
# DONT RUN THIS

# Evaluate the model
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
print(f"Mean Squared Error: {mse}")

Mean Squared Error: [0.0305876  0.03341802 0.03520555 0.07742365]


In [ ]:
encoded_data.to_csv('encoded_data.csv', index=False)

In [ ]:
from google.colab import files

files.download('encoded_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
merged_data.to_csv('encoded_data.csv', index=False)

In [ ]:
from joblib import dump

# Assuming 'model' is your trained scikit-learn model (e.g., pipeline or estimator)
dump(model, 'olympics_data_model.joblib')

['olympics_data_model.joblib']

In [ ]:
from google.colab import files

files.download('olympics_data_model.joblib')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
host_country_data = merged_data.copy()
host_country_data_filtered = host_country_data[host_country_data['Medal'] != 'No Medals'].copy()
host_countries_list = list(host_countries[1])
host_countries

KeyError: 1